
Importar librerias<br>


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import shap  # Instalar la librerÃ­a SHAP si no estÃ¡ instalada
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')


S6.T0. Importar la base de datos a PyCharm<br>
ïƒ˜ Guarda la base de datos BBDD_ST5.csv en un dataframe llamado "df"<br>
ïƒ˜ El dtype de las columnas indicadas a continuaciÃ³n las convertimos de float64/int64 a float32/int32 para reducir memoria<br>
ïƒ˜ Separa los inputs/output en set de entrenamiento (train: 80 %) y validaciÃ³n (test: 20 %) con la funciÃ³n train_test_split.<br>


In [ ]:
df = pd.read_csv(r'E:\Laptop\BBDD_ST5.csv')  # Importar la base de datos BBDD_ST5.csv

In [ ]:
float64_cols = df.select_dtypes(include='float64').columns.tolist()  # Seleccionar las columnas de tipo float64
df[float64_cols] = df[float64_cols].astype('float32')  # Convertir las columnas de tipo float64 a float32
int64_cols = df.select_dtypes(include='int64').columns.tolist()  # Seleccionar las columnas de tipo int64
df[int64_cols] = df[int64_cols].astype('int32')  # Convertir las columnas de tipo int64 a int32

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop("QualificaciÃ³ de consum d'energia primaria no renovable", axis=1),  # Inputs
    df["QualificaciÃ³ de consum d'energia primaria no renovable"],  # Output
    test_size=0.2,  # TamaÃ±o del set de validaciÃ³n (20 %)
    random_state=1)  # Semilla para controlar la aleatoriedad. No tocar


S6.T1. ConstrucciÃ³n de un clasificador basado en "ANN con softmax activation"<br>
ïƒ˜ Consigue entrenar un modelo ANN con tus datos (df).<br>
ïƒ˜ Compara los resultados del "clasification report" de tu modelo en el test de validaciÃ³n con los valores de referencia:<br>
               precision    recall  f1-score   support<br>
           0       0.56      0.57      0.56     32389<br>
           1       0.00      0.00      0.00     18259<br>
           2       0.65      0.85      0.74     75197<br>
           3       0.30      0.29      0.30     13190<br>
           4       0.34      0.08      0.13      5191<br>
           5       0.59      0.27      0.37      1143<br>
           6       0.53      0.20      0.29       355<br>
    accuracy                           0.60    145724<br>
   macro avg       0.42      0.32      0.34    145724<br>
CÃ³mo variable resumen del comportamiento de vuestro modelo, os podÃ©is fijar en el accuracy, que en este caso es del 57 %.<br>



Puedes modificar estas variables para evaluar distintas configuraciones de la red neuronal
<br>
neurons_hidden_layers = (10, 20)  # NÃºmero de neuronas en cada capa. En este caso: 10 neuronas en la primera capa y 10 neuronas en la segunda capa.<br>
max_iter = 20  # NÃºmero de iteraciones para el entrenamiento del modelo (epochs).<br>
class_weight = {0: 1.63, 1: 2.91, 2: 0.7, 3: 4.05, 4: 10.3, 5: 44.3, 6: 142.2}  # Para compensar el desbalance de clases.<br>
in de la variables a modificar



Crear el modelo de red neuronal
<br>
mlp = MLPClassifier(<br>
    hidden_layer_sizes=neurons_hidden_layers,<br>
    activation='relu',<br>
    solver='adam',<br>
    max_iter=max_iter,<br>
    learning_rate_init=0.00005,<br>
    random_state=1234,<br>
    verbose=1  # Muestra detalles durante el entrenamiento<br>
)<br>
ntrenar el modelo con el set de entrenamiento


In [ ]:
history = mlp.fit(x_train, y_train)  # Entrenar el modelo con el set de entrenamiento


Visualizar mÃ©tricas del entrenamiento.
<br>
train_loss = history.loss_curve_<br>
# Calcular accuracy por epoch en el set de entrenamiento<br>
train_accuracy = []<br>
for i in range(len(train_loss)):<br>
    y_train_pred = mlp.predict(x_train)<br>
    acc = accuracy_score(y_train, y_train_pred)<br>
    train_accuracy.append(acc)<br>
# Graficar el rendimiento por epoch<br>
epochs = range(1, len(train_loss) + 1)<br>
plt.figure(figsize=(12, 6))<br>
# GrÃ¡fico de la pÃ©rdida (loss)<br>
plt.subplot(1, 2, 1)<br>
plt.plot(epochs, train_loss, label='Training Loss', color='blue')<br>
plt.xlabel('Epochs')<br>
plt.ylabel('Loss')<br>
plt.title('Training Loss per Epoch')<br>
plt.grid(alpha=0.3)<br>
plt.legend()<br>
# GrÃ¡fico del accuracy<br>
plt.subplot(1, 2, 2)<br>
plt.plot(epochs, train_accuracy, label='Training Accuracy', color='green')<br>
plt.xlabel('Epochs')<br>
plt.ylabel('Accuracy')<br>
plt.title('Training Accuracy per Epoch')<br>
plt.grid(alpha=0.3)<br>
plt.legend()<br>
plt.tight_layout()<br>
plt.show()<br>
valuar el modelo con el set de validaciÃ³n. Imprimir el accuracy para el set de entrenamiento y validaciÃ³n. 


<br>
y_pred = mlp.predict(x_test)  # Predecir el set de validaciÃ³n<br>
print(classification_report(y_test, y_pred))  # Imprimir el classification report<br>
print(classification_report(y_train, mlp.predict(x_train)))  # Imprimir el classification report<br>
onfusion matrix on test set

    
<br>
cm = confusion_matrix(y_test, y_pred)<br>
plt.figure(figsize=(10, 7))<br>
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["G", "F", "E", "D", "C", "B", "A"], yticklabels=["G", "F", "E", "D", "C", "B", "A"])<br>
plt.xlabel('Predicted Labels')<br>
plt.ylabel('True Labels')<br>
plt.title('Confusion Matrix')<br>
plt.show()<br>
HAP values